In [ ]:
2+3

5

In [7]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
import matplotlib.pyplot as plt

def estimate_price(data_path, input_date):
    # Load the dataset
    df = pd.read_csv(data_path)

    # Ensure 'Dates' column is in datetime format
    df['Dates'] = pd.to_datetime(df['Dates'])

    # Extract the month and set it as the index
    df['Months'] = df['Dates'].dt.to_period('M')
    df.set_index('Months', inplace=True)

    # Drop the original 'Dates' column
    df.drop(columns=['Dates'], inplace=True)

    # Fit the SARIMA model
    model = SARIMAX(df['Prices'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit()

    # Predict price on a given date
    input_date = pd.to_datetime(input_date, format='%Y-%m-%d').to_period('M')
    if input_date in df.index:
        return df.loc[input_date, 'Prices']
    else:
        last_date = df.index[-1]
        forecast_steps = (input_date.year - last_date.year) * 12 + (input_date.month - last_date.month)
        forecast = model_fit.get_forecast(steps=forecast_steps)
        forecast_index = pd.date_range(start=last_date.to_timestamp(), periods=forecast_steps + 1, freq='M')[1:].to_period('M')
        forecast_df = pd.DataFrame({'Forecast': forecast.predicted_mean}, index=forecast_index)
        if input_date in forecast_df.index:
            return forecast_df.loc[input_date, 'Forecast']
        else:
            return np.nan

def calculate_contract_value(data_path, injection_dates, withdrawal_dates, injection_rate, withdrawal_rate, max_storage_volume, storage_cost_per_unit):
    # Predict prices for all injection and withdrawal dates
    injection_prices = [estimate_price(data_path, date) for date in injection_dates]
    withdrawal_prices = [estimate_price(data_path, date) for date in withdrawal_dates]

    # Calculate total injection cost
    total_injection_cost = sum([price * injection_rate for price in injection_prices])

    # Calculate total withdrawal revenue
    total_withdrawal_revenue = sum([price * withdrawal_rate for price in withdrawal_prices])

    # Calculate storage costs
    total_storage_volume = min(injection_rate * len(injection_dates), max_storage_volume)
    total_storage_cost = total_storage_volume * storage_cost_per_unit * len(set(injection_dates + withdrawal_dates))

    # Calculate net value of the contract
    net_value = total_withdrawal_revenue - total_injection_cost - total_storage_cost

    return net_value

# Define input parameters
data_path = 'Nat_Gas.csv'
injection_dates = ['2023-01-01', '2023-02-01', '2023-03-01']
withdrawal_dates = ['2023-09-01', '2023-10-01', '2023-11-01']
injection_rate = 1000  # units of gas per injection
withdrawal_rate = 1000  # units of gas per withdrawal
max_storage_volume = 5000  # maximum storage capacity in units of gas
storage_cost_per_unit = 0.50  # storage cost per unit of gas per month

# Calculate the net value of the contract
net_value = calculate_contract_value(data_path, injection_dates, withdrawal_dates, injection_rate, withdrawal_rate, max_storage_volume, storage_cost_per_unit)
print(f'Net value of the contract: ${net_value:.2f}')


Net value of the contract: $-9300.00


In [3]:
import warnings
warnings.filterwarnings("ignore")